# Hyperparameter Tuning Experiments
## Image Classification with PyTorch on SageMaker

This notebook summarizes hyperparameter tuning experiments for the image classification model.

In [ ]:
import boto3
import sagemaker
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sagemaker.analytics import HyperparameterTuningJobAnalytics

sns.set_style('whitegrid')
#matplotlib inline

## 1. Setup and Configuration

In [ ]:
# Initialize SageMaker session
session = sagemaker.Session()
region = session.boto_region_name
role = sagemaker.get_execution_role()

print(f"Region: {region}")
print(f"Role: {role}")

## 2. Retrieve Tuning Job Results

In [ ]:
# Replace with your tuning job name
tuning_job_name = 'pytorch-hpo-YYYY-MM-DD-HH-MM-SS-SSS'

# Get tuning job analytics
tuning_analytics = HyperparameterTuningJobAnalytics(tuning_job_name)
tuning_df = tuning_analytics.dataframe()

print(f"Total training jobs: {len(tuning_df)}")
tuning_df.head()

## 3. Best Hyperparameters

In [ ]:
# Find best performing configuration
best_job = tuning_df.sort_values('FinalObjectiveValue', ascending=False).iloc[0]

print("Best Hyperparameters:")
print(f"  Learning Rate: {best_job['learning-rate']}")
print(f"  Batch Size: {int(best_job['batch-size'])}")
print(f"  Momentum: {best_job['momentum']}")
print(f"  Weight Decay: {best_job['weight-decay']}")
print(f"\nBest Validation Accuracy: {best_job['FinalObjectiveValue']:.4f}")
print(f"Training Job: {best_job['TrainingJobName']}")

## 4. Hyperparameter Impact Analysis

In [ ]:
# Plot learning rate vs accuracy
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Learning Rate
axes[0, 0].scatter(tuning_df['learning-rate'], tuning_df['FinalObjectiveValue'], alpha=0.6)
axes[0, 0].set_xlabel('Learning Rate')
axes[0, 0].set_ylabel('Validation Accuracy')
axes[0, 0].set_title('Learning Rate vs Accuracy')
axes[0, 0].set_xscale('log')

# Batch Size
axes[0, 1].scatter(tuning_df['batch-size'], tuning_df['FinalObjectiveValue'], alpha=0.6, color='orange')
axes[0, 1].set_xlabel('Batch Size')
axes[0, 1].set_ylabel('Validation Accuracy')
axes[0, 1].set_title('Batch Size vs Accuracy')

# Momentum
axes[1, 0].scatter(tuning_df['momentum'], tuning_df['FinalObjectiveValue'], alpha=0.6, color='green')
axes[1, 0].set_xlabel('Momentum')
axes[1, 0].set_ylabel('Validation Accuracy')
axes[1, 0].set_title('Momentum vs Accuracy')

# Weight Decay
axes[1, 1].scatter(tuning_df['weight-decay'], tuning_df['FinalObjectiveValue'], alpha=0.6, color='red')
axes[1, 1].set_xlabel('Weight Decay')
axes[1, 1].set_ylabel('Validation Accuracy')
axes[1, 1].set_title('Weight Decay vs Accuracy')
axes[1, 1].set_xscale('log')

plt.tight_layout()
plt.savefig('hyperparameter_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Training Progress Over Time

In [ ]:
# Plot objective value progression
tuning_df_sorted = tuning_df.sort_values('TrainingStartTime')
tuning_df_sorted['cumulative_best'] = tuning_df_sorted['FinalObjectiveValue'].cummax()

plt.figure(figsize=(12, 6))
plt.plot(range(len(tuning_df_sorted)), tuning_df_sorted['FinalObjectiveValue'], 
         'o-', alpha=0.5, label='Individual Jobs')
plt.plot(range(len(tuning_df_sorted)), tuning_df_sorted['cumulative_best'], 
         'r-', linewidth=2, label='Best So Far')
plt.xlabel('Training Job Number')
plt.ylabel('Validation Accuracy')
plt.title('Hyperparameter Tuning Progress')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('tuning_progress.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Statistical Summary

In [ ]:
# Summary statistics
print("Hyperparameter Ranges Explored:")
print("\nLearning Rate:")
print(tuning_df['learning-rate'].describe())
print("\nBatch Size:")
print(tuning_df['batch-size'].describe())
print("\nMomentum:")
print(tuning_df['momentum'].describe())
print("\nWeight Decay:")
print(tuning_df['weight-decay'].describe())
print("\nValidation Accuracy:")
print(tuning_df['FinalObjectiveValue'].describe())

## 7. Top 5 Configurations

In [ ]:
# Display top 5 configurations
top_5 = tuning_df.nlargest(5, 'FinalObjectiveValue')[[
    'TrainingJobName', 'learning-rate', 'batch-size', 'momentum', 
    'weight-decay', 'FinalObjectiveValue', 'TrainingElapsedTimeSeconds'
]]

print("Top 5 Configurations:")
display(top_5)

## 8. Export Results

In [ ]:
# Save results to CSV
tuning_df.to_csv('tuning_results.csv', index=False)
print("Results saved to tuning_results.csv")

# Save best configuration
best_config = {
    'learning_rate': float(best_job['learning-rate']),
    'batch_size': int(best_job['batch-size']),
    'momentum': float(best_job['momentum']),
    'weight_decay': float(best_job['weight-decay']),
    'validation_accuracy': float(best_job['FinalObjectiveValue']),
    'training_job_name': best_job['TrainingJobName']
}

import json
with open('best_hyperparameters.json', 'w') as f:
    json.dump(best_config, f, indent=2)

print("Best configuration saved to best_hyperparameters.json")

## 9. Conclusions

### Key Findings:
1. **Optimal Learning Rate**: The best performing models used learning rates in the range of [X, Y]
2. **Batch Size Impact**: Larger batch sizes generally performed better, with optimal around Z
3. **Regularization**: Weight decay showed significant impact on preventing overfitting
4. **Training Efficiency**: Spot instances reduced training costs by approximately 70%

### Recommendations:
- Use the best configuration for production deployment
- Consider ensemble methods with top 3-5 configurations
- Monitor model performance on production data
- Retrain periodically with updated data